# FlickBike - Distributed Processing (DIP)
---

Voor deze opdracht werken we samen met FlickBike. FlickBike is een aanbieder van deelfietsen. Dat zijn fietsen die voor korte termijn, meestal een enkele rit, gehuurd kunnen worden door abonnees. In het geval van FlickBike mag je een fiets huren en achterlaten waar je maar wilt.

---

### Verkenning van de analyse: vragen

---
- __1 Hoe bereken je  de groei van FlickBike? In het bijzonder: hoe voorspel je het aantal verhuurde fietsen op de dagen tussen 10 december 2019 en 31 januari 2020?__ 
    - Data begint op 1 januari 2019
    - Data eindigt op 9 december 2019
    - Door een correlatie test uit te voeren kun je erachter komen of er bepaalde parameters zijn die een sterke correlatie hebben
    - Door lineaire regressie toe te passen zou je voor de toekomstige dagen kunnen voorspellen hoeveel fietsen er zullen worden gehuurd op basis van de oude datapunten.
    
    
    

---

- __2 Welke invloeden verwacht je van, en op welke manier kun je je model gevoelig maken voor:__ 
     - __*Het weer.*__
         - Weer heeft veel invloed op het gebruik van deze FlickBikes omdat mensen natuurlijk met slecht weer, denk hierbij aan regen, minder snel/helemaal niet de fiets nemen. Als het goed weer is heeft het bijvoorbeeld wel een positief effect en nemen mensen sneller de fiets. Hoewel ik met alle weersomstandigheden het niet aanraad om te fietsen in Amsterdam. 
     - __*De dag van de week.*__
         - Doordeweeks is een groot deel van de mensen aan het werk waardoor deze fietsen niet voor persoonlijk gebruik worden gebruikt. In het weekend zijn meer mensen vrij.  Dan kun je zeggen, ze gaan op stap en nemen een huurfiets, of ze blijven juist thuis. 
     - __*Bijzondere dagen in het jaar, zoals feestdagen, schoolvakanties, etc.*__
         - Met bepaalde feestdagen, denk aan Pasen, zijn mensen niet snel buiten omdat ze met familie thuis zijn. Zo is dat ook vaker met andere feestdagen omdat buiten de winkels ook dicht zijn etc.  
     - __*Eventueel andere factoren.*__
         - Als benzineprijzen omhoog springen gaan mensen minder snel tanken, en nemen eerder alternatief vervoer. 

---

- __3 Hoe kan, op basis van data, worden voorspeld waar er tekorten van fietsen dreigen te ontstaan en hoe de fietsen het beste zo kunnen worden verplaatst dat ze zowel over het hele terrein van Schiphol-Oost beschikbaar zijn en tegelijkertijd vooral beschikbaar zijn op plaatsen waar het regelmatig druk is?__  

    - Ten eerste kijk je waarde 'hotspots' zijn, dus waar de fietsen het meest worden gebruikt.
    - Bekijk hiervoor in welke buurt de 'start-lan' parameter het meeste voorkomt.
    - Ten tweede kijk je waarde de minst drukke plekken zijn waar 'start-lan' voorkomt. Het is handig om op deze plekken juist fietsen beschikbaar te stellen
    - Doe dit hetzelfde voor de 'end-lan'
---
- __4 Hoe kun je berekenen met hoeveel fietsen zou FlickBike minimaal kunnen werken, zodanig dat te verwachten pieken op te vangen zijn?__

    - Bekijk met de huidige dataset op welke dag het 't drukst was maar ook wat het gemiddelde per dag is. Door iets boven het gemiddelde uit te komen zorg je ervoor dat de meeste mensen sowieso in bezit kunnen zijn van een huurfiets
---

- __5 Hoe ga je de antwoorden op vragen (1)-(4) visualiseren? Geef voorbeelden!__
    - vraag 1: per dag (x-as) aantal verhuurde fietsen (y-as) visualiseren. 
    - vraag 2: correlatie test uitvoeren (data.corr())
    - vraag 3: een interactieve map waarin alle fietsen zichtbaar zijn 
    - vraag 4: simpele plot zoals vraag 1 

#### Data dictionary

---

- __Account__ = business
- __Bikeid__ = intern gebruik
- __Bikenumber__ = zichtbaar op fietsen (QR-CODE)
- __Date__ = moment van invoering in database
- __Start_time__ = start van sessie
- __End_time__ = einde van sessie

---
- __test.csv__ = fietsverhuur tussen 1-11-2019 en 20-12-2019
- __train.csv__ = fietsverhuur voor 1-11-2019. Wordt gebruikt om voorspellingen te maken voor de dagen die hierop volgen
---

#### Import libraries 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.graphics.tsaplots import plot_acf
import pandas as pd
import seaborn as sns

#### Load data

In [ ]:
df = pd.read_csv(("/Users/wolfsinem/distributed-processing/bike-rental-flickbike/data/bikes.csv"))

In [ ]:
df.head()

#### Convert column date and start/end time to datetime

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])

In [ ]:
type(list(df['date'].values)[0])

### Opdracht: Data van fietsverhuur
---

#### Vraag 1: produceer een plot van verhuur-per-dag voor deze dataset

In [ ]:
# variable consists of index(date) and values(amount)
group_date = df.groupby(df.date.dt.date)["tripid"].sum() # tripid or bikeid etc?? 

# plot rental per day
plt.figure(figsize=(20,10))
sns.barplot(group_date.index, group_date.values, alpha=0.8)
plt.title('Rental FlickBike per day')
plt.ylabel('Number of rentals', fontsize=12)
plt.xlabel('Day', fontsize=12)
# plt.locator_params(axis='x', nbins=12) # tick values will be places incorrectly, fix this
plt.show()

In [ ]:
plt.plot(group_date.index, group_date.values)
plt.show()

#### Vraag 2: wat is de periodiciteit van deze dataset?

In [ ]:
plot_acf(group_date.values)
plt.show()

###### onderbouwing plot

#### Correlation between all columns

In [ ]:
sns.heatmap(df.corr())
plt.show()

#### Add external data for holidays (2019)

In [ ]:
national_holidays = pd.read_csv(("/Users/wolfsinem/distributed-processing/bike-rental-flickbike/data/Nationale Feestdagen.csv"))
other_holidays = pd.read_csv(("/Users/wolfsinem/distributed-processing/bike-rental-flickbike/data/Overige Feestdagen.csv"))

In [ ]:
all_holidays = pd.concat([national_holidays,other_holidays]).reset_index(drop=True)
all_holidays['Datum'] = pd.to_datetime(all_holidays['Datum'])

In [ ]:
all_holidays.head()

#### Merge holidays with original dataset 'df'

In [ ]:
df['holiday'] = df['date'].dt.date.isin(all_holidays) # change datetime

In [ ]:
# df['date'] = df.date.dt.strftime('%Y-%m-%d')

#### Add new column to dataframe with total duration

In [ ]:
start_time = df['start_time']
end_time = df['end_time']

delta_duration = end_time - start_time
df['total_duration'] = delta_duration
# returns duration in days instead of hours/mins/seconds --> have to change later

In [ ]:
plt.plot(start_time)

#### Question 1: predict the growth of FlickBike between 2019/12/10 and 2020/1/31

In [ ]:
# total_duration = []

# for i in delta_duration:
#     seconds = i.total_seconds()
# #      minutes = (seconds % 3600) // 60
#     seconds = seconds % 60
